<b>Exercise 2: Running the NEURON with Python interface. Adaptive vs. fixed time step solution methods.</b>
<br><br>
We use the original Hodgkin-Huxley model
$$C_{\rm m}\frac{{\rm d}V}{{\rm d}t} = g_{\rm Na} m^3h(E_{\rm Na}-V) + g_{\rm K} n^4(E_{\rm K}-V) + g_{\rm l}(E_{\rm L}-V) + I_{\rm app}$$
where $m$ is controlled by:
$$ \frac{{\rm d}m}{{\rm d}t} = \alpha_m (1-m)-\beta_m m$$
$h$ is controlled by:
$$ \frac{{\rm d}h}{{\rm d}t} = \alpha_h (1-h)-\beta_h h$$
and $n$ is controlled by:
$$ \frac{{\rm d}n}{{\rm d}t} = \alpha_n (1-n)-\beta_n n$$
The Hodgkin-Huxley mechanism is present in the somatic section. In addition, we add dendritic segments where only passive current ($I_{\rm L} = g_{\rm l}(E_{\rm L}-V)$) is present.

In [1]:
#Import the python libraries
import matplotlib.pyplot as plt
plt.ion()
from neuron import h
import numpy
import time
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#Run NEURON with Python interface
def runneuron(isadaptive=False,amp=0.12,dur=10):
    my_start_time = time.time()
    h.load_file("stdlib.hoc")                        #A default NEURON library
    h.load_file("stdrun.hoc")                        #A default NEURON library

    h('objref cvode')                                #Define an object cvode
    h('cvode = new CVode()')                         #Make cvode a time step integrator object
    h('cvode.active('+str(isadaptive*1)+')')         #Set the variable time step integration on/off

    h('create soma, dend[3]')                        #Create soma and three dendritic sections

    h.dend[0].connect(h.soma,1,0)                    #Connect the sections to each other
    h.dend[1].connect(h.soma,1,0)
    h.dend[2].connect(h.dend[0],1,0)

    h('access soma')                                 #Make soma the currently accessed section

    for sec in [h.dend[0], h.dend[1], h.dend[2]]:
        sec.insert('pas')                            #Insert passive membrane mechanism to all sections
        sec.nseg = 5                                 #and use 5 sub-compartments for each section
    h.soma.insert('hh')                              #Insert the Hodgkin-Huxley mechanism (includes leak) to soma
    h.soma.nseg = 5                                  

    #Set the passive parameters for each section:
    h.soma.Ra = 100; h.soma.cm = 1; h.soma.diam = 10; h.soma.L = 20;
    h.dend[0].Ra = 100; h.dend[0].cm = 1; h.dend[0].diam = 5; h.dend[0].L = 200; h.dend[0].e_pas = -54.3; h.dend[0].g_pas = 0.00003
    h.dend[1].Ra = 100; h.dend[1].cm = 1; h.dend[1].diam = 2; h.dend[1].L = 100; h.dend[1].e_pas = -54.3; h.dend[1].g_pas = 0.00003
    h.dend[2].Ra = 100; h.dend[2].cm = 1; h.dend[2].diam = 2; h.dend[2].L = 160; h.dend[2].e_pas = -54.3; h.dend[2].g_pas = 0.00003

    h.dt = 0.025                                     #Set the time step to 0.025 ms
    h.tstop = 200                                    #Continue the simulation until 200 ms

    h('objref stim, Vrec')                           #Declare objects stim and Vrec

    h.stim = h.IClamp(0.5, sec = h.soma)             #Make stim an IClamp object, stimulating center (0.5) of soma
    h.stim.delay = 100                               #Stimulation starts at 100 ms
    h.stim.dur = dur                                  #and lasts 10 ms
    h.stim.amp = amp                                 #and has a given amplitude (default 0.075 nA)

    Vrec = h.Vector()                                #Record somatic membrane potential and time
    Vrec.record(h.soma(0.5)._ref_v, sec=h.soma)
    trec = h.Vector()
    trec.record(h._ref_t)

    h.init()                                         #Initialize state variables
    h.run()                                          #Run the simulations


    plt.plot(trec,Vrec)
    plt.xlabel('$t$ (ms)')
    plt.xlim([0, 200])
    plt.ylim([-80, 40])
    plt.ylabel('$V_m$ (mV)')
    plt.title("Simulation run in "+str(time.time() - my_start_time)+" seconds.")
    

In [4]:
slider = widgets.interact(runneuron, isadaptive=widgets.Checkbox(value=False,description='Use adaptive time step method',layout=Layout(width='50%')),
                          amp=widgets.FloatSlider(min=0.0,max=0.3,step=0.01,value=0.12,description='Amplitude of square pulse current: amp',layout=Layout(width='50%'), continuous_update = False),
                          dur=widgets.FloatSlider(min=0.0,max=60,step=2,value=10,description='Duration of square pulse current: dur',layout=Layout(width='50%'), continuous_update = False))


interactive(children=(Checkbox(value=False, description='Use adaptive time step method', layout=Layout(width='…

#Question 1: What is the threshold current for the somatic DC stimulation?

#Question 2: What is the threshold amplitude for a 6-ms somatic current clamp?

#Question 3a: Can you make the adaptive and fixed time step solvers show notably different behaviour? <i>Hint: try to find a threshold amplitude for the fourth spike using maximum duration.</i>
#***Question 3b***: Change the code above to make the fixed time step solver more accurate. Does the difference you found in 3a completely disappear?